In [0]:
-- dbfs:/mnt/demo-datasets/bookstore
use catalog hive_metastore;

## Reading Stream

In [0]:
select * from books limit 3;

In [0]:
describe detail books

In [0]:
%fs ls "dbfs:/user/hive/warehouse/books"

In [0]:
optimize books

In [0]:
describe detail books

In [0]:
%fs ls "dbfs:/user/hive/warehouse/books"

In [0]:
describe history books

In [0]:
select count(*) from books;
select * from books limit 3;

In [0]:
%python
spark.readStream.table("books").createOrReplaceTempView("books_streaming_tmp_vw")

## Displaying Streaming Data

In [0]:
select * from books_streaming_tmp_vw limit 3

In [0]:
insert into books values ("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 ),("B10", "Beginning Database Design Solutions","Danny","Computer Science",11 )

## Applying Transformations

In [0]:
select author, count(book_id) as total_books
from books_streaming_tmp_vw
GROUP BY author

## Unsupported Operations

In [0]:
 SELECT * 
 FROM books_streaming_tmp_vw
 ORDER BY author

## Persisting Streaming Data

In [0]:
create or replace temp view author_counts_tmp_vw as (
  select author, count(book_id) as total_books
  FROM books_streaming_tmp_vw
  group by author
)

In [0]:
select * from author_counts_tmp_vw;

In [0]:
%fs ls "dbfs:/mnt/demo/author_counts_checkpoint"

In [0]:
%python
(spark.table("author_counts_tmp_vw")
    .writeStream    # 이 데이터는 배치(batch) 가 아니라 스트리밍 데이터라는 것을 Spark에 알려줌
    .trigger(processingTime='4 seconds')   # 새 데이터가 있든 없든 4초마다 한 번씩 처리
    .outputMode("complete") # 매번 실행할 때마다 전체 결과를 다시 계산해서 전부 다시 씀 / 보통 COUNT, SUM 같은 집계 결과에 사용
    .option("checkpointLocation","dbfs:/mnt/demo/author_counts_checkpoint") # 스트리밍의 생명줄, 어디까지 처리했는지 기억, 장애 발생 시 그 지점부터 재시작
    .table("author_counts") # 결과를 Databricks 테이블로 저장
)

In [0]:
select * from author_counts

In [0]:
%fs ls "dbfs:/mnt/demo/author_counts_checkpoint"

In [0]:
%fs ls "dbfs:/mnt/demo/author_counts_checkpoint/__tmp_path_dir/"

In [0]:

%fs head "dbfs:/mnt/demo/author_counts_checkpoint/metadata"


In [0]:
describe extended author_counts

In [0]:
%fs ls "dbfs:/user/hive/warehouse/author_counts"

In [0]:
describe history author_counts

## Adding New Data

In [0]:
INSERT INTO books
values ("B19", "Introduction to Modeling and Simulation", "Mark W. Spong", "Computer Science", 25),
        ("B20", "Robot Modeling and Control", "Mark W. Spong", "Computer Science", 30),
        ("B21", "Turing's Vision: The Birth of Computer Science", "Chris Bernhardt", "Computer Science", 35)

## Streaming in Batch Mode

In [0]:
select * from author_counts

In [0]:
INSERT INTO books
values ("B16", "Hands-On Deep Learning Algorithms with Python", "Danny", "Computer Science", 25),
        ("B17", "Neural Network Methods in Natural Language Processing", "Danny", "Computer Science", 30),
        ("B18", "Understanding digital signal processing", "Richard Lyons", "Computer Science", 35)

In [0]:
%python
# 지금까지 쌓여 있는 모든 스트리밍 데이터를 한 번에 전부 처리하고 결과를 테이블에 저장한 뒤 작업을 끝내는 코드
(spark.table("author_counts_tmp_vw")  # 이미 만들어진 임시 뷰를 읽음                               
      .writeStream                    # DataFrame은 스트리밍 데이터라는 선언
      .trigger(availableNow=True)     # 지금까지 들어와 있는 데이터만 전부 처리 끝나면 스트리밍 종료
      .outputMode("complete")         # 매번 실행할 때마다 집계 결과 전체를 다시 계산 / 보통 COUNT / SUM 같은 집계에 사용
      .option("checkpointLocation", "dbfs:/mnt/demo/author_counts_checkpoint")
      .table("author_counts")       
      .awaitTermination()              # 이 스트리밍 작업이 완전히 끝날 때까지 기다림/ availableNow에서는 모든 데이터 처리완료 →자동 종료→awaitTermination()끝남
)

In [0]:
-- (spark.table("author_counts_tmp_vw")
--     .writeStream    # 이 데이터는 배치(batch) 가 아니라 스트리밍 데이터라는 것을 Spark에 알려줌
--     .trigger(processingTime='4 seconds')   # 새 데이터가 있든 없든 4초마다 한 번씩 처리
--     .outputMode("complete") # 매번 실행할 때마다 전체 결과를 다시 계산해서 전부 다시 씀 / 보통 COUNT, SUM 같은 집계 결과에 사용
--     .option("checkpointLocation","dbfs:/mnt/demo/author_counts_checkpoint") # 스트리밍의 생명줄, 어디까지 처리했는지 기억, 장애 발생 시 그 지점부터 재시작
--     .table("author_counts") # 결과를 Databricks 테이블로 저장
-- )

In [0]:
select * from author_counts